In [1]:
from sys import path
path.append('..')
path.append('../../src/')

import pickle as pkl
from os.path import join, exists

import numpy as np
import matplotlib.pyplot as plt

from icao_dataset import ICAODataset

from global_config import IMAGE_SIZE
from local_config import FOLDER_MODELS, FILE_DATASET

%matplotlib inline

In [2]:
PATH_TO_ROOT = '../../'

FOLDER_MODELS = join(PATH_TO_ROOT, FOLDER_MODELS)
FILE_DATASET = join(PATH_TO_ROOT, FILE_DATASET)

print(FOLDER_MODELS)
print(FILE_DATASET)

../../models/landmark_decoder/
../../data/cropped_faces_eyes/dataset_landmark_decoder.pkl


# Data Loading 

In [3]:
if exists(FILE_DATASET):
    print("Dataset found. Loading...")
    dataset = pkl.load(open(FILE_DATASET, "rb"))
else:
    print("Generating dataset...")
    dataset = ICAODataset()
    dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=(160, 160))
    pkl.dump(dataset, open(FILE_DATASET, "wb"))

Dataset found. Loading...


In [4]:
%%time

new_dataset = ICAODataset()
new_dataset.y_images = []
n_files = len(dataset.image_files)

for i, (file, im, mrk) in enumerate(dataset):
    print(f"{i + 1} of {n_files}", end="\r")
    if mrk.photo_reqs.dark_tinted_lenses.is_non_compliant():
        continue
    
    y_eye_right = np.array(mrk.right_eye.center().as_list()).astype(np.int)
    y_eye_left = np.array(mrk.left_eye.center().as_list()).astype(np.int)

    im_landmark = np.zeros((*IMAGE_SIZE, 1))
    im_landmark[y_eye_left[1], y_eye_left[0]] = 255
    im_landmark[y_eye_right[1], y_eye_right[0]] = 255
    
    new_dataset.image_files.append(file)
    new_dataset.images.append(im)
    new_dataset.y_images.append(im_landmark.astype(np.uint8))
    new_dataset.mrks.append(mrk)

Wall time: 970 ms


In [5]:
len(new_dataset.image_files)

4999

In [6]:
FILE_OUTPUT = '../../data/cropped_faces_eyes/dataset_landmark_decoder.pkl'

pkl.dump(new_dataset, open(FILE_OUTPUT, "wb"), protocol=4)